### Shapash Model Overview
https://shapash.readthedocs.io/en/latest/

##### With this tutorial you:
Understand how to create a Shapash SmartPredictor to make prediction and have local explanation in production with a simple use case.

This tutorial describes the different steps from training the model to Shapash SmartPredictor deployment. A more detailed tutorial allows you to know more about the SmartPredictor Object.

Contents:

- Build a Regressor
- Compile Shapash SmartExplainer
- From Shapash SmartExplainer to SmartPredictor
- Save Shapash Smartpredictor Object in pickle file
- Make a prediction

#Example 1


Video walkthrough of the following code : https://www.youtube.com/watch?v=-66Wt3IHb9U

In [1]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
### Divide the dataset into independent and dependent dataset
y=df['tip']
X=df[df.columns.difference(['tip'])]
X.head()

,day,sex,size,smoker,time,total_bill
0,Sun,Female,2,No,Dinner,16.99
1,Sun,Male,3,No,Dinner,10.34
2,Sun,Male,3,No,Dinner,21.01
3,Sun,Male,2,No,Dinner,23.68
4,Sun,Female,4,No,Dinner,24.59


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.3 KB


In [4]:
X['day']=X['day'].cat.codes
X['sex']=X['sex'].cat.codes
X['smoker']=X['smoker'].cat.codes
X['time']=X['time'].cat.codes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [5]:
X

,day,sex,size,smoker,time,total_bill
0,3,1,2,1,1,16.99
1,3,0,3,1,1,10.34
2,3,0,3,1,1,21.01
3,3,0,2,1,1,23.68
4,3,1,4,1,1,24.59
...,...,...,...,...,...,...
239,2,0,3,1,1,29.03
240,2,1,2,0,1,27.18
241,2,0,2,0,1,22.67
242,2,0,2,1,1,17.82


In [6]:
### Train Test split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=1)
regressor = RandomForestRegressor(n_estimators=200).fit(X_train,y_train)

#### Lets Understand Our Model With Shapash 
In this section, we use the SmartExplainer Object from shapash.

- It allows users to understand how the model works with the specified data.
- This object must be used only for data mining step. Shapash provides another object for deployment.


In [ ]:
!pip install shapash

In [8]:
from shapash.explainer.smart_explainer import SmartExplainer
xpl = SmartExplainer()
xpl.compile(
    x=X_test,
    model=regressor,
    
)

xpl

Backend: Shap TreeExplainer


#### Lets Understand the results of your trained model
Then, we can easily get a first summary of the explanation of the model results.

- Here, we chose to get the 3 most contributive features for each prediction.
- We used a wording to get features names more understandable in operationnal case.

In [11]:
app = xpl.run_app(title_story='Tips Dataset')

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Dash is running on http://0.0.0.0:8050/



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://47a62ae84b0a:8050/
INFO:root:Use the method .kill() to down your app.
INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)


In [12]:
xpl.to_pandas(max_contrib=3).head()

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
67,1.48755,total_bill,3.07,-1.0928,day,2,-0.263028,size,1,-0.1198
243,2.47635,total_bill,18.78,-0.2365,day,0,-0.111353,smoker,1,-0.108559
206,3.48225,total_bill,26.59,0.468972,size,3,0.0953703,sex,0,-0.0383996
122,2.22030,total_bill,14.26,-0.518983,day,0,-0.152207,size,2,-0.0422378
89,3.18580,total_bill,21.16,0.390149,day,0,-0.0893861,size,2,-0.0257139


In [ ]:
predictor = xpl.to_smartpredictor()

In [ ]:
predictor.save('./predictor.pkl')

In [ ]:
from shapash.utils.load_smartpredictor import load_smartpredictor
predictor_load = load_smartpredictor('./predictor.pkl')

#### Make a prediction with your SmartPredictor
In order to make new predictions and summarize local explainability of your model on new datasets, you can use the method add_input of the SmartPredictor.

- The add_input method is the first step to add a dataset for prediction and explainability.
- It checks the structure of the dataset, the prediction and the contribution if specified.
- It applies the preprocessing specified in the initialisation and reorder the features with the order used by the model. (see the documentation of this method)
- In API mode, this method can handle dictionnaries data which can be received from a GET or a POST request.
- Add data
- The x input in add_input method doesn't have to be encoded, add_input applies preprocessing.

In [ ]:
predictor_load.add_input(x=X, ypred=y)

In [ ]:
detailed_contributions = predictor_load.detail_contributions()


In [ ]:
detailed_contributions.head()

,tip,day,sex,size,smoker,time,total_bill
0,1.01,0.058890,0.107123,-0.021913,-0.032742,-0.005985,0.183477
1,1.66,0.083062,-0.058770,0.003739,-0.062743,-0.018027,-1.230111
2,3.50,0.084890,-0.021616,-0.005301,-0.008053,0.006988,0.428840
3,3.31,0.125816,-0.021922,-0.043787,0.022929,0.007135,0.074177
4,3.61,0.048222,0.058356,-0.053819,-0.006406,-0.006721,-0.319232


#### Summarize explanability of the predictions
- You can use the summarize method to summarize your local explainability
- This summary can be configured with modify_mask method so that you have explainability that meets your operational needs.

In [ ]:
predictor_load.modify_mask(max_contrib=3)

In [ ]:
explanation = predictor_load.summarize()

In [ ]:
explanation.head()

,tip,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
0,1.01,total_bill,16.99,0.183477,sex,1.0,0.107123,day,3.0,0.05889
1,1.66,total_bill,10.34,-1.230111,day,3.0,0.083062,smoker,1.0,-0.062743
2,3.50,total_bill,21.01,0.42884,day,3.0,0.08489,sex,0.0,-0.021616
3,3.31,day,3.0,0.125816,total_bill,23.68,0.074177,size,2.0,-0.043787
4,3.61,total_bill,24.59,-0.319232,sex,1.0,0.058356,size,4.0,-0.053819


#Example 2

https://github.com/MAIF/shapash/blob/master/tutorial/tutorial01-Shapash-Overview-Launch-WebApp.ipynb

In [12]:
!pip install category_encoders

     |████████████████████████████████| 81kB 4.9MB/s 


In [13]:
import pandas as pd
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [16]:
from shapash.data.data_loader import data_loading
house_df, house_dict = data_loading('house_prices')

y_df=house_df['SalePrice'].to_frame()
X_df=house_df[house_df.columns.difference(['SalePrice'])]

house_df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2-Story 1946 & Newer,Residential Low Density,8450,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,Normal,Normal,Single-family Detached,Two story,7,5,2003,2003,Gable,Standard (Composite) Shingle,Vinyl Siding,Vinyl Siding,Brick Face,196.0,Good,Average/Typical,Poured Contrete,Good (90-99 inches),Typical - slight dampness allowed,No Exposure/No Basement,Good Living Quarters,706,Unfinished/No Basement,0,150,856,Gas forced warm air furnace,Excellent,Yes,Standard Circuit Breakers & Romex,856,854,0,1710,1,0,2,1,3,1,Good,8,Typical Functionality,0,Attached to home,2003.0,Rough Finished,548,Typical/Average,Typical/Average,Paved,0,61,0,0,0,0,0,2,2008,Warranty Deed - Conventional,Normal Sale,208500
2,1-Story 1946 & Newer All Styles,Residential Low Density,9600,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Frontage on 2 sides of property,Gentle slope,Veenker,Adjacent to feeder street,Normal,Single-family Detached,One story,6,8,1976,1976,Gable,Standard (Composite) Shingle,Metal Siding,Metal Siding,None,0.0,Average/Typical,Average/Typical,Cinder Block,Good (90-99 inches),Typical - slight dampness allowed,Good Exposure,Average Living Quarters,978,Unfinished/No Basement,0,284,1262,Gas forced warm air furnace,Excellent,Yes,Standard Circuit Breakers & Romex,1262,0,0,1262,0,1,2,0,3,1,Typical/Average,6,Typical Functionality,1,Attached to home,1976.0,Rough Finished,460,Typical/Average,Typical/Average,Paved,298,0,0,0,0,0,0,5,2007,Warranty Deed - Conventional,Normal Sale,181500
3,2-Story 1946 & Newer,Residential Low Density,11250,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,Normal,Normal,Single-family Detached,Two story,7,5,2001,2002,Gable,Standard (Composite) Shingle,Vinyl Siding,Vinyl Siding,Brick Face,162.0,Good,Average/Typical,Poured Contrete,Good (90-99 inches),Typical - slight dampness allowed,Mimimum Exposure,Good Living Quarters,486,Unfinished/No Basement,0,434,920,Gas forced warm air furnace,Excellent,Yes,Standard Circuit Breakers & Romex,920,866,0,1786,1,0,2,1,3,1,Good,6,Typical Functionality,1,Attached to home,2001.0,Rough Finished,608,Typical/Average,Typical/Average,Paved,0,42,0,0,0,0,0,9,2008,Warranty Deed - Conventional,Normal Sale,223500
4,2-Story 1945 & Older,Residential Low Density,9550,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Corner lot,Gentle slope,Crawford,Normal,Normal,Single-family Detached,Two story,7,5,1915,1970,Gable,Standard (Composite) Shingle,Wood Siding,Wood Shingles,None,0.0,Average/Typical,Average/Typical,Brick & Tile,Typical (80-89 inches),Good,No Exposure/No Basement,Average Living Quarters,216,Unfinished/No Basement,0,540,756,Gas forced warm air furnace,Good,Yes,Standard Circuit Breakers & Romex,961,756,0,1717,1,0,1,0,3,1,Good,7,Typical Functionality,1,Detached from home,1998.0,Unfinished/No Garage,642,Typical/Average,Typical/Average,Paved,0,35,272,0,0,0,0,2,2006,Warranty Deed - Conventional,Abnormal Sale,140000
5,2-Story 1946 & Newer,Residential Low Density,14260,Paved,Slightly irregular,Near F

In [17]:
from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df=encoder.transform(X_df)

Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=1)
regressor = LGBMRegressor(n_estimators=200).fit(Xtrain,ytrain)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [18]:
from shapash.explainer.smart_explainer import SmartExplainer 
xpl = SmartExplainer(features_dict=house_dict) # optional parameter, specifies label for features name

xpl.compile(
    x=Xtest,
    model=regressor,
    preprocessing=encoder # Optional: compile step can use inverse_transform method
)

app = xpl.run_app(title_story='House Prices')

Backend: Shap TreeExplainer


INFO:numexpr.utils:NumExpr defaulting to 2 threads.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



Dash is running on http://0.0.0.0:8050/



INFO:root:Your Shapash application run on http://47a62ae84b0a:8050/
INFO:root:Use the method .kill() to down your app.
INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)
 * Environment: production


In [19]:
app.kill()

In [20]:
summary_df= xpl.to_pandas(
    max_contrib=3, # Number Max of features to show in summary
    threshold=5000,
)
xpl.save('./xpl.pkl')

In [21]:
summary_df.head()

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
259,209141.256921,Ground living area square feet,1792,13710.4,Overall material and finish of the house,7,12776.3,Total square feet of basement area,963,-5103.03
268,178734.474531,Ground living area square feet,2192,29747,Overall material and finish of the house,5,-26151.3,Overall condition of the house,8,9190.84
289,113950.844570,Overall material and finish of the house,5,-24730,Ground living area square feet,900,-16342.6,Total square feet of basement area,882,-5922.64
650,74957.162142,Overall material and finish of the house,4,-33927.7,Ground living area square feet,630,-23234.4,Total square feet of basement area,630,-11687.9
1234,135305.243500,Overall material and finish of the house,5,-25445.7,Ground living area square feet,1188,-11476.6,Condition of sale,Abnormal Sale,-5071.82
